# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

# Exploring fenaroli parsing

([A-Z]{3,50}\s?[A-Z]{3,50}\s?[A-Z]{3,50}) # original re for the flavor compounds

## Loading plaintext generated by slate (on AWS instance)

In [11]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)
with open('../data/doc_split.pkl', 'r') as f:
    split = pickle.load(f)

In [132]:
# cleaned_joined = joined.replace('\n\n', 'zxzxzxzx').replace('\n',' ').replace('.', ' ')
cleaned_joined = joined.replace('.', ' ')

In [255]:
cleaned_joined[:600000]

entries I'm interested in start at split doc 17

## RE for flavor compounds

In [93]:
flavor_search_string = '([\w-]?-?[A-Z]{2,20}[\s,-|]?\s?[0-9]?,?[0-9]?[\s-]?[A-Z]{2,50}[,-|\s]?[A-Z]{2,20})'
sqfc = re.compile(flavor_search_string)
raw_flavor_compounds = re.findall(sqfc, joined)
terms_to_remove = ['FEMA PADI','EINECS', 'EMA PADI', 'EINECS NO','FEMA PAD','JECFAN', 'FEMA.PADI']
flavor_comp = [comp for comp in raw_flavor_compounds if comp not in terms_to_remove and comp[:2] != '-C']

In [94]:
flavor_series = pd.Series(flavor_comp)
top_terms = list(flavor_series.value_counts().index.values[:10])

In [95]:
print len(flavor_comp)
flavor_comp[:1000]

In [16]:
flavor_series.value_counts()

In [21]:
cnt = Counter(flavor_comp)
to_remove = [k for k, v in cnt.iteritems() if v > 2]

In [23]:
to_remove

In [66]:
top_terms[:10]

In [17]:
flavor_compounds = [compound.replace('\n','') for compound in raw_flavor_compounds]

In [18]:
flavor_comp = [comp for comp in flavor_compounds if comp not in to_remove and comp[:2] != '-C']

In [85]:
len(to_remove)

In [74]:
_ = flavor_series.value_counts().hist(bins = 20, log =True)

## RE for ingredients

This is the ultimate regex pattern. It finds the natural occurence, and searches below for the next flavor compound
'Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+'

In [292]:
sq = re.compile('Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+')
raw_ingredients = re.findall(sq, cleaned_joined)

ingredient_series = pd.Series(raw_ingredients)

In [293]:
len(raw_ingredients)

In [294]:
raw_ingredients[:100]

In [90]:
stop_strings = ingredient_series.value_counts()[:10].index[:4]

top_terms = list(ingredient_series.value_counts().index.values[:10])
stop_strings=top_terms[:4]+ top_terms[5:7] + [top_terms[8]]

In [91]:
cleaned_ingredients = [comp.strip() for comp in raw_ingredients if comp not in stop_strings]
cleaned_ingredients[:10]